# DNN approach for maRAPID

Goodsol Lee, Netlab, Seoul National University

## Get datasets

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os

#configuration for gpu usage
conf = tf.ConfigProto()
# you can modify below as you want
#conf.gpu_options.per_process_gpu_memory_fraction = 0.4
#conf.gpu_options.allow_growth = True
os.environ['CUDA_VISIBLE_DEVICES']='0'
print(tf.__version__)

1.12.0


<font size="5">Sample training data, validation data, test data from raw data</font> 
data is classified with its cell id

In [17]:
node_num = 155
cell_num = 7
num_RSSI_sample = 10

data_set = np.empty((cell_num, num_RSSI_sample))
channel_label_set = np.empty((cell_num, 1))
TA_label_set = np.empty((cell_num, 1))

rssi_sample_set =[[] for _ in range(cell_num)]
channel_sample_set = [[] for _ in range(cell_num)]
TA_sample_set = [[] for _ in range(cell_num)]

for node_index in range(1,node_num+1):
    print('Get sample from node'+str(node_index))
    file_name = 'data/node-'+str(node_index)+'.txt'
    previous_cellId = -1
    cell_sample = list()
    
    with open(file_name, 'r') as f:
        while 1:
            line = f.readline()
            if not line: break
            parsed_line =line.split(' ')
            
            rssi = parsed_line[9]
            channel_state = parsed_line[8]
            current_cellId = parsed_line[10]
            TA_region = parsed_line[11]
            
            if previous_cellId == current_cellId:
                cell_sample.append([rssi, channel_state, TA_region])
            else:
                previous_cellId = current_cellId
                cellId = int(previous_cellId)-1
                print("Cell: "+str(cellId+1))
                if cell_sample:
                    cell_sample = np.array(cell_sample)
                    for j in range(len(cell_sample)-num_RSSI_sample+1):
                        rssi_sample_set[cellId].append(cell_sample[j:j+num_RSSI_sample,0])
                        channel_sample_set[cellId].append(cell_sample[j+num_RSSI_sample-1,1])
                        TA_sample_set[cellId].append(cell_sample[j+num_RSSI_sample-1,2])
                
                cell_sample = list()
        
        cell_sample = np.array(cell_sample)
        for j in range(len(cell_sample)-num_RSSI_sample+1):
            rssi_sample_set[cellId].append(cell_sample[j:j+num_RSSI_sample,0])
            channel_sample_set[cellId].append(cell_sample[j+num_RSSI_sample-1,1])
            TA_sample_set[cellId].append(cell_sample[j+num_RSSI_sample-1,2])
        
        sample_length = 0
        for i in range(len(channel_sample_set)):
            sample_length += len(rssi_sample_set[i])-num_RSSI_sample+1
        
        print(sample_length)

sample_length = 0        
for i in range(len(channel_sample_set)):
    cell_length = len(channel_sample_set[i])
    print("Cell"+str(i+1)+" sample length is "+str(cell_length))
    sample_length += cell_length
    
print('Sampling is completed, sample length: ',sample_length)

Get sample from node1
Cell: 7
Cell: 1
Cell: 5
Cell: 1
Cell: 4
25690
Get sample from node2
Cell: 1
Cell: 5
41273
Get sample from node3
Cell: 7
Cell: 1
Cell: 5
Cell: 1
Cell: 4
66633
Get sample from node4
Cell: 7
Cell: 1
85215
Get sample from node5
Cell: 5
Cell: 1
Cell: 6
111787
Get sample from node6
Cell: 1
Cell: 7
Cell: 1
132970
Get sample from node7
Cell: 6
Cell: 7
Cell: 1
Cell: 5
163543
Get sample from node8
Cell: 5
Cell: 1
Cell: 7
Cell: 6
216508
Get sample from node9
Cell: 6
Cell: 1
243089
Get sample from node10
Cell: 4
Cell: 1
Cell: 2
Cell: 1
Cell: 3
264868
Get sample from node11
Cell: 2
Cell: 1
Cell: 4
Cell: 1
Cell: 5
286420
Get sample from node12
Cell: 1
Cell: 5
Cell: 1
Cell: 5
303182
Get sample from node13
Cell: 5
Cell: 1
Cell: 6
336553
Get sample from node14
Cell: 7
Cell: 1
Cell: 5
359931
Get sample from node15
Cell: 5
Cell: 1
Cell: 7
Cell: 6
412093
Get sample from node16
Cell: 6
Cell: 1
Cell: 5
446866
Get sample from node17
Cell: 6
Cell: 7
Cell: 1
473238
Get sample from node18


3929479
Get sample from node148
Cell: 5
Cell: 1
Cell: 7
3951651
Get sample from node149
Cell: 1
Cell: 6
Cell: 1
3980222
Get sample from node150
Cell: 1
Cell: 5
Cell: 1
Cell: 7
4005583
Get sample from node151
Cell: 6
Cell: 7
Cell: 1
4024954
Get sample from node152
Cell: 7
Cell: 1
4038135
Get sample from node153
Cell: 1
Cell: 5
Cell: 1
4051306
Get sample from node154
Cell: 5
Cell: 1
4058487
Get sample from node155
Cell: 4
4059478
Cell1 sample length is 1482014
Cell2 sample length is 141060
Cell3 sample length is 102727
Cell4 sample length is 68973
Cell5 sample length is 712174
Cell6 sample length is 716232
Cell7 sample length is 836361
Sampling is completed, sample length:  4059541


In [18]:
my_sample=[ [] for _ in range(cell_num)]
sample = (1,2,4,5,6)
print(my_sample)
my_sample[1].append(sample)
print(my_sample)

[[], [], [], [], [], [], []]
[[], [(1, 2, 4, 5, 6)], [], [], [], [], []]


In [19]:
len(rssi_sample_set[0])

1482014

In [20]:
#choose cell number-1 to train and test
desired_cell = 0

data_set = np.array(rssi_sample_set[desired_cell])
channel_label_set = np.array(np.expand_dims(channel_sample_set[desired_cell],axis=1))
TA_label_set = np.array(np.expand_dims(TA_sample_set[desired_cell],axis=1))

print(channel_label_set)

[['0']
 ['0']
 ['1']
 ...
 ['1']
 ['0']
 ['1']]


In [21]:
import random

data_length = data_set.shape[0]
valid_num = (int)(data_length/10)
test_num = (int)(data_length/10)

#get test set
test_index = random.sample(range(0,data_length-test_num),test_num)
test_data = data_set[test_index]
test_channel_label = channel_label_set[test_index]
test_TA_label = TA_label_set[test_index]

#get training set/validation set
train_data_set = np.delete(data_set,test_index,axis=0)
train_channel_label_set = np.delete(channel_label_set,test_index,axis=0)
train_TA_label_set = np.delete(TA_label_set,test_index,axis=0)
data_length = train_data_set.shape[0]

valid_index = random.sample(range(0,data_length),valid_num)

training_data = train_data_set
training_channel_label = train_channel_label_set
training_TA_label = train_TA_label_set

valid_data = train_data_set[valid_index]
valid_channel_label = train_channel_label_set[valid_index]
valid_TA_label = train_TA_label_set[valid_index]

save_data ={
    'training_data':training_data,
    'training_channel_label':training_channel_label,
    'training_TA_label':training_TA_label,
    
    'valid_data':valid_data,
    'valid_channel_label':valid_channel_label,
    'valid_TA_label':valid_TA_label,
    
    'test_data':test_data,
    'test_channel_label':test_channel_label,
    'test_TA_label':test_TA_label
}

In [22]:
# get name of the data path
data_path = 'data/save_data_cell'+str(desired_cell)+'_sample'+str(num_RSSI_sample)+'.pickle'

In [23]:
# save data
with open(data_path,'wb') as f:
    pickle.dump(save_data,f,pickle.HIGHEST_PROTOCOL)

In [24]:
# restore data
with open(data_path,'rb') as f:
    save = pickle.load(f)
    
    train_dataset = save['training_data']
    train_channel_labels = save['training_channel_label']
    train_labels = save['training_TA_label']
    
    valid_dataset = save['valid_data']
    valid_channel_labels = save['valid_channel_label']
    valid_labels = save['valid_TA_label']
    
    test_dataset = save['test_data']
    test_channel_labels = save['test_channel_label']
    test_labels = save['test_TA_label']
    
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (1333813, 10) (1333813, 1)
Validation set (148201, 10) (148201, 1)
Test set (148201, 10) (148201, 1)


In [25]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, num_RSSI_sample)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    enc.fit(labels)
    labels = enc.transform(labels).toarray()
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this wa

Training set (1333813, 10) (1333813, 4)
Validation set (148201, 10) (148201, 4)
Test set (148201, 10) (148201, 4)


/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [28]:
sample_size = num_RSSI_sample
num_labels = 4

layer1_output_num = 100
layer2_output_num = 100
layer3_output_num = 1024
layer4_output_num = 512
layer5_output_num = 256
num_steps = 1000
batch_size = (int)(data_length/20)

graph_gs=tf.Graph()
with graph_gs.as_default():
    tf_dataset_gs=tf.placeholder(tf.float32, shape=(None, sample_size))
    tf_labels_gs=tf.placeholder(tf.float32, shape=(None, num_labels))
    keep_prob = tf.placeholder(tf.float32, shape=(None))
    
    lambda_reg = 0.0000000001
        
    #Regularization
    regularizer = tf.contrib.layers.l2_regularizer(scale=lambda_reg)
    initializer = tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)
    #neural network consists of two lines
    dense1 = tf.layers.dense(tf_dataset_gs, layer1_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense1 = tf.nn.dropout(dense1, keep_prob = keep_prob)
    dense2 = tf.layers.dense(dense1,layer2_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense2 = tf.nn.dropout(dense2, keep_prob = keep_prob)
    dense3 = tf.layers.dense(dense2 ,layer3_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense3 = tf.nn.dropout(dense3, keep_prob = keep_prob)
    dense4 = tf.layers.dense(dense3 ,layer4_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense4 = tf.nn.dropout(dense4, keep_prob = keep_prob)
    dense5 = tf.layers.dense(dense4 ,layer4_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense5 = tf.nn.dropout(dense5, keep_prob = keep_prob)
    logits_gs = tf.layers.dense(dense2, num_labels, activation=None)
    
    #Loss
    loss_gs = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_labels_gs, logits=logits_gs))
    
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.005
    
    decay_num =  50*(int)(train_labels.shape[0]/batch_size)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_num, 1, staircase=True)
    # Optimizer
    optimizer_gs = tf.train.AdamOptimizer(learning_rate).minimize(loss_gs, global_step=global_step)
    
    #Predictions for the training
    prediction_gs = tf.nn.softmax(logits_gs)

In [ ]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.equal(np.argmax(predictions, 1), np.argmax(labels, 1)))
          / predictions.shape[0])

tf.reset_default_graph()
batch_num = (int)(train_labels.shape[0]/batch_size)

with tf.Session(graph=graph_gs, config=conf) as session_gs:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        random = np.random.permutation(train_labels.shape[0])
        loss = 0
        for b in range(batch_num):
            batch_data = train_dataset[random[b*batch_size:(b+1)*batch_size]]
            batch_labels = train_labels[random[b*batch_size:(b+1)*batch_size]].astype(float)
            feed_dict_gs = {tf_dataset_gs: batch_data, tf_labels_gs: batch_labels, keep_prob:1}
            _, l_gs, predictions_l = session_gs.run([optimizer_gs, loss_gs, prediction_gs], feed_dict=feed_dict_gs)
            loss += l_gs
        print("Minibatch loss at step %d: %f" % (step, loss))
        feed_dict_val_gs = {tf_dataset_gs: valid_dataset}
        valid_prediction_gs = session_gs.run(prediction_gs, feed_dict={tf_dataset_gs: valid_dataset, tf_labels_gs: valid_labels, keep_prob:1})
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction_gs, valid_labels))
        
    feed_dict_test_gs = {tf_dataset_gs: test_dataset, keep_prob:1.0}
    test_prediction_gs = session_gs.run(prediction_gs, feed_dict=feed_dict_test_gs)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction_gs, test_labels))
    saver = tf.train.Saver()
    saver.save(session_gs, "./model_checkpoints/distributed_cellId"+str(desired_cell+1)+"_sample"+str(sample_size))

Initialized
Minibatch loss at step 0: 350.027778
Validation accuracy: 31.7%
Minibatch loss at step 1: 71.417807
Validation accuracy: 31.9%
Minibatch loss at step 2: 31.258496
Validation accuracy: 36.8%
Minibatch loss at step 3: 25.977800
Validation accuracy: 40.2%
Minibatch loss at step 4: 24.471329
Validation accuracy: 41.1%
Minibatch loss at step 5: 23.853517
Validation accuracy: 43.3%
Minibatch loss at step 6: 23.518214
Validation accuracy: 44.2%
Minibatch loss at step 7: 23.325683
Validation accuracy: 44.7%
Minibatch loss at step 8: 23.191377
Validation accuracy: 45.1%
Minibatch loss at step 9: 23.079434
Validation accuracy: 45.7%
Minibatch loss at step 10: 22.985977
Validation accuracy: 45.5%
Minibatch loss at step 11: 22.900380
Validation accuracy: 45.8%
Minibatch loss at step 12: 22.803655
Validation accuracy: 46.0%
Minibatch loss at step 13: 22.721306
Validation accuracy: 46.8%
Minibatch loss at step 14: 22.641304
Validation accuracy: 46.7%
Minibatch loss at step 15: 22.551392


In [93]:
with tf.Session(graph=graph_gs, config=conf) as session_gs:
    tf.global_variables_initializer().run()
    feed_dict_test_gs = {tf_dataset_gs: test_dataset, keep_prob:1}
    test_prediction_gs = session_gs.run(prediction_gs, feed_dict=feed_dict_test_gs)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction_gs, test_labels))

Test accuracy: 52.5%
